In [1]:
learning_rate = 0.01
weight_decay = 0.0001
batch_size = None
num_epochs = 50


In [2]:
import numpy as np
image = np.load("Image_data_per_episode.npy",mmap_mode='c')
episode_length = np.load("Episode.npy",mmap_mode='c')
velocity = np.load("Velocity.npy",mmap_mode='c')
r_theta_data = np.load("R_T_data.npy",mmap_mode='c')


test_image = np.load("test_Image.npy",mmap_mode='c')
test_episode_length = np.load("test_Episode.npy",mmap_mode='c')
test_velocity = np.load("test_velocity.npy",mmap_mode='c')
test_r_theta_data = np.load("test_R_T_data.npy",mmap_mode='c')


In [3]:
from tensorflow.keras.layers import Dropout,Dense,Flatten,Input,Activation,Concatenate,Conv2D,Flatten,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.applications import VGG16,MobileNet
from tensorflow.keras.optimizers import Adam
import numpy as np
import random
from collections import deque
from math import atan

#baseModel = VGG16(weights="imagenet", include_top=False,
#                input_tensor=Input(shape=(224, 224, 3)))

In [4]:
episode_length_list = []
for i in episode_length:
    episode_length_list.append(int(i))
    

test_episode_length_list = []
for i in test_episode_length:
    test_episode_length_list.append(int(i))

In [5]:
episode_indices = np.arange(len(episode_length))

# Shuffle the episode indices
np.random.shuffle(episode_indices)



test_episode_indices = np.arange(len(test_episode_length))

# Shuffle the episode indices
np.random.shuffle(test_episode_indices)

In [6]:

# Split the Image array into episodes based on the episode_lengths
episodes = np.split(image, episode_length_list)[:-1]

# Rearrange the episodes based on the shuffled episode_indices
shuffled_episodes = [episodes[i] for i in episode_indices]

# Concatenate the shuffled episodes back into a single array
image = np.concatenate(shuffled_episodes, axis=0)





episodes = np.split(test_image, test_episode_length_list)[:-1]

# Rearrange the episodes based on the shuffled episode_indices
shuffled_episodes = [episodes[i] for i in test_episode_indices]

# Concatenate the shuffled episodes back into a single array
test_image = np.concatenate(shuffled_episodes, axis=0)

In [7]:

# Split the Image array into episodes based on the episode_lengths
episodes = np.split(velocity, episode_length_list)[:-1]

# Rearrange the episodes based on the shuffled episode_indices
shuffled_episodes = [episodes[i] for i in episode_indices]

# Concatenate the shuffled episodes back into a single array
velocity = np.concatenate(shuffled_episodes, axis=0)





episodes = np.split(test_velocity, test_episode_length_list)[:-1]

# Rearrange the episodes based on the shuffled episode_indices
shuffled_episodes = [episodes[i] for i in test_episode_indices]

# Concatenate the shuffled episodes back into a single array
test_velocity = np.concatenate(shuffled_episodes, axis=0)

In [8]:

# Split the Image array into episodes based on the episode_lengths
episodes = np.split(r_theta_data, episode_length_list)[:-1]

# Rearrange the episodes based on the shuffled episode_indices
shuffled_episodes = [episodes[i] for i in episode_indices]

# Concatenate the shuffled episodes back into a single array
r_theta_data = np.concatenate(shuffled_episodes, axis=0)





episodes = np.split(test_r_theta_data, test_episode_length_list)[:-1]

# Rearrange the episodes based on the shuffled episode_indices
shuffled_episodes = [episodes[i] for i in test_episode_indices]

# Concatenate the shuffled episodes back into a single array
test_r_theta_data = np.concatenate(shuffled_episodes, axis=0)

In [9]:
pip install tensorflow_addons


[notice] A new release of pip available: 22.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from tensorflow.keras.layers import Activation , concatenate
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#pip install tensorflow-addons
import tensorflow_addons as tfa

/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.2 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're us

In [11]:
input_shape = (160, 120, 3)

x_train = image
y_train = velocity

x_test = test_image
y_test = test_velocity
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")


x_train shape: (25603, 160, 120, 3) - y_train shape: (25603, 2, 3)


In [12]:
def MLP(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


In [13]:
base_model = MobileNet(
    input_shape=(160,120,3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights='mobilenet_1_0_224_tf_no_top.h5',
    input_tensor=None,

)




for layer in base_model.layers:
    layer.trainable = True    #!!!!!!!

2023-06-12 16:02:18.772511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-12 16:02:18.813410: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-12 16:02:18.813747: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-12 16:02:18.815255: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [14]:
def feature_ext():
        image_inputs = layers.Input(shape=(160,120,3)) #shape=(None, 160, 120, 3))
        goal_inputs = layers.Input(shape=(2,))
            # Augment data.

        g = Dense(32,activation = 'linear')(goal_inputs)
        g = Dense(128,activation = 'linear')(g)
        g = Dense(512,activation = 'linear')(g)
        g = Dense(2048,activation = 'linear')(g)
        g = Dense(8192,activation = 'linear')(g)
        g = Dense(19200,activation = 'relu')(g)
        print("kir to hossein")
        g= tf.reshape(g,(-1,160,120,1))
        #g=tf.expand_dims(g,axis=0)

        conCat = Concatenate()([image_inputs,g])
        inp = Conv2D(3,3, padding='same', activation="relu")(conCat)
        
        
        kir_to= base_model(inp)


        #inp=tf.expand_dims(inp,axis=0)
        

        features = Flatten()(kir_to)


        
        linear_vel = Dense(4096, activation="relu",name = 'first_lin')(features)
        linear_vel = Dense(2048, activation="relu")(linear_vel)
        linear_vel = Dense(1024, activation="relu")(linear_vel)
        linear_vel = Dense(512, activation="relu")(linear_vel)
        linear_vel = Dense(128, activation="relu")(linear_vel)
        linear_vel = Dense(32, activation="relu")(linear_vel)
        linear_vel = Dense(1,)(linear_vel)                        
        linear_net =Activation("linear", name="linear_output")(linear_vel)                  
                                                                                        
                                                                                         
        angular_vel = Dense(4096, activation="relu",name = 'first_ang')(features)
        angular_vel = Dense(2048, activation="relu")(angular_vel)
        angular_vel = Dense(1024, activation="relu")(angular_vel)
        angular_vel = Dense(512, activation="relu")(angular_vel)
        angular_vel = Dense(128, activation="relu")(angular_vel)
        angular_vel = Dense(32, activation="relu")(angular_vel)
        angular_vel = Dense(1,)(angular_vel)                        
        angular_net =Activation("linear", name="angular_output")(angular_vel)   

        
        
        
        model = Model(inputs=[image_inputs,goal_inputs], outputs=[linear_net,angular_net])

        
        return model


In [15]:
feature = feature_ext()
feature.summary()

kir to hossein
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 32)           96          ['input_3[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 128)          4224        ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 512)          66048       ['dense_1[0][0]']                
                                                                               

In [17]:
def scheduler(epoch, lr):
    if epoch%2 ==0:
        return lr * tf.math.exp(-0.1)
    else:
        return lr
def run_experiment(model):
    lr_planer = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )
#######################################################################################################################33    
    losses={'linear_output':'mse','angular_output':'mse'}                      #########################################
    weight={'linear_output':1,'angular_output':4}                                                                                   ################################
    model.compile(
        optimizer=optimizer,
        loss=losses,
        metrics={ 'linear_output':"mean_squared_error",'angular_output': "mean_squared_error"},loss_weights=weight)
    
    log_dir = ''
    checkpoint_filepath = log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
    checkpoint_callback = keras.callbacks.ModelCheckpoint( 
        #callback is an object that can perform actions at various stages of training
        checkpoint_filepath,
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=[x_train,r_theta_data],
        y=[y_train[:,0,0],y_train[:,1,2]],
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=(
            [x_test,test_r_theta_data],
            [y_test[:,0,0],y_test[:,1,2]]
        ),
        callbacks=[checkpoint_callback,lr_planer],
        shuffle=False
    )

    model.load_weights(checkpoint_filepath)
 #   _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
  #  print(f"Test accuracy: {round(accuracy * 100, 2)}%")
   # print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")
    
    model.save('')
    return history




In [ ]:
freture = feature_ext()
history = run_experiment(freture)

kir to hossein


2023-06-12 16:04:01.892351: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5898931200 exceeds 10% of free system memory.
2023-06-12 16:04:06.128794: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5898931200 exceeds 10% of free system memory.


Epoch 1/50


2023-06-12 16:04:13.934394: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-12 16:04:14.109297: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2023-06-12 16:04:15.518719: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-06-12 16:04:15.519854: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-06-12 16:04:15.519913: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-06-12 16:04:15.521301: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-06-12 16:04:15.521416: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to

801/801 [==============================] - 73s 83ms/step - loss: 28534732.0000 - linear_output_loss: 22926686.0000 - angular_output_loss: 1402013.1250 - linear_output_mean_squared_error: 22926686.0000 - angular_output_mean_squared_error: 1402013.1250 - val_loss: 0.3834 - val_linear_output_loss: 0.0143 - val_angular_output_loss: 0.0923 - val_linear_output_mean_squared_error: 0.0143 - val_angular_output_mean_squared_error: 0.0923 - lr: 0.0090
Epoch 2/50
801/801 [==============================] - 63s 78ms/step - loss: 0.3366 - linear_output_loss: 0.0089 - angular_output_loss: 0.0819 - linear_output_mean_squared_error: 0.0089 - angular_output_mean_squared_error: 0.0819 - val_loss: 0.3802 - val_linear_output_loss: 0.0140 - val_angular_output_loss: 0.0915 - val_linear_output_mean_squared_error: 0.0140 - val_angular_output_mean_squared_error: 0.0915 - lr: 0.0067
Epoch 9/50
697/801 [=========================>....] - ETA: 7s - loss: 0.3381 - linear_output_loss: 0.0084 - angular_output_loss: 0.0